In [ ]:
# Show plots inline, and load main getdist plot module and samples class
from __future__ import print_function
#%matplotlib inline
#%config InlineBackend.figure_format = 'retina'
import matplotlib as mpl
import sys, os
sys.path.insert(0,os.path.realpath(os.path.join(os.getcwd(),'..')))
from getdist import plots, MCSamples, loadMCSamples
from getdist.gaussian_mixtures import GaussianND
import getdist
# use this *after* importing getdist if you want to use interactive plots
# %matplotlib notebook
import matplotlib.pyplot as plt
import IPython
import numpy as np
import pandas as pd
import cosmicfish as cf
print('GetDist Version: %s, Matplotlib version: %s'%(getdist.__version__, plt.matplotlib.__version__))
# matplotlib 2 may not work very well without usetex on, can uncomment
# plt.rcParams['text.usetex']=True

#mpl.rcParams['agg.path.chunksize'] = 1000000000000

fullmasses = np.geomspace(0.01, 10., 21)
#massidxs =  np.array([0, 14, 20])
massidxs = np.array([0])
masses = np.array(fullmasses[mval] for midx, mval in enumerate(massidxs))

survey = 'CMBS4'

lssfishers = [
    np.loadtxt(
        '/Users/nicholasdeporzio/Desktop/ellipse_plots/'+survey+'/T0.91/M'+str(mval+1)+'_LSS_Fisher.dat', 
        delimiter='\t', 
        skiprows=1)
    for midx, mval in enumerate(massidxs)]
fullfishers = [
    np.loadtxt(
        '/Users/nicholasdeporzio/Desktop/ellipse_plots/'+survey+'/T0.91/M'+str(mval+1)+'_Full_Fisher.dat', 
        delimiter='\t', 
        skiprows=1)
    for midx, mval in enumerate(massidxs)]
cmbfishers = [
    np.loadtxt(
        '/Users/nicholasdeporzio/Desktop/ellipse_plots/'+survey+'/T0.91/Fisher'+survey+'_bin'+str(mval+1)+'_T0.91K.txt', 
        delimiter='\t', 
        skiprows=1)
    for midx, mval in enumerate(massidxs)]

fullcovs = [np.linalg.inv(fval) for fidx, fval in enumerate(fullfishers)]
lsscovs = [np.linalg.inv(np.delete(np.delete(fval, 5, axis=0), 5, axis=1)) for fidx, fval in enumerate(lssfishers)]
cmbcovs = [np.linalg.inv(fval) for fidx, fval in enumerate(cmbfishers)]

for midx, mval in enumerate(massidxs):
    lsscovs[midx] = np.insert(lsscovs[midx], 5, 0, axis=0)
    lsscovs[midx] = np.insert(lsscovs[midx], 5, 0, axis=1)
    lsscovs[midx][5,5]=1.0e15 #FUDGING TAU FOR LSS ONLY SURVEY

fullfid = np.array([2.2321e-9, 0.701, 0.967, 0.02226, 0.1127, 0.0598, 1., 250., 1.,  1.])
lssfid = np.array([2.2321e-9, 0.701, 0.967, 0.02226, 0.1127, 0.0598, 1., 250., 1.,  1.])
cmbfid = np.array([2.2321e-9, 0.701, 0.967, 0.02226, 0.1127, 0.0598, 1.])

fullnames = [r'$A_s$', r'$h$', r'$n_s$', r'$\omega_b$', r'$\omega_{cdm}$', r'$\tau_{reio}$', r'$g_{\chi}$', r'$\sigma_{fog}$', r'$\beta_0$', r'$\alpha_{k2}$']
lssnames = [r'$A_s$', r'$h$', r'$n_s$', r'$\omega_b$', r'$\omega_{cdm}$', r'$\tau_{reio}$', r'$g_{\chi}$', r'$\sigma_{fog}$', r'$\beta_0$', r'$\alpha_{k2}$']
cmbnames = [r'$A_s$', r'$h$', r'$n_s$', r'$\omega_b$', r'$\omega_{cdm}$', r'$\tau_{reio}$', r'$g_{\chi}$']

#fullnames = [r'$A_s$', r'$h$', r'$n_s$', r'$\omega_b$', r'$\omega_{cdm}$', r'$\tau_{reio}$', r'$g_{\chi}$', r'$\sigma_{fog}$', r'$\beta_0$', r'$\alpha_{k2}$']
#lssnames = [r'A_s', r'h', r'n_s', r'omega_b', r'omega_cdm', r'N_ncdm', r'sigma_fog', r'b0', r'alpha_k2']
#cmbnames = [r'A_s', r'h', r'n_s', r'omega_b', r'omega_cdm', r'tau_reio', r'N_ncdm']


full_gauss = [GaussianND(fullfid, cval, names=fullnames) for cidx, cval in enumerate(fullcovs)]
lss_gauss = [GaussianND(lssfid, cval, names=lssnames) for cidx, cval in enumerate(lsscovs)]
cmb_gauss = [GaussianND(cmbfid, cval, names=cmbnames) for cidx, cval in enumerate(cmbcovs)]

for midx, mval in enumerate(massidxs):
    h = plots.get_subplot_plotter()
    # Set a default plot style 
    from getdist.styles.tab10 import style_name
    plots.set_active_style(style_name)
    # Adjust plot sizes, styles, etc using getdist.plots.GetDistPlotSettings class
    h.settings.figure_legend_frame = False #Box around legend
    h.settings.scaling=False #Defaults to font size for Tick Labels: 11, Labels: 14, Legend: 12, Text: 12
    h.settings.axes_fontsize=14
    h.settings.axes_labelsize=20
    h.settings.axis_tick_x_rotation=45
    h.settings.legend_fontsize=20
    #h.settings.axis_tick_y_rotation=90
    #h.settings.colorbar_tick_rotation=90
    h.triangle_plot(
        [cmb_gauss[midx], lss_gauss[midx], full_gauss[midx]], 
        filled=False,
        legend_labels=[survey+' Only', 'DESI Only', 'DESI+'+survey],
        line_args=
            [
                {'ls':'dotted', 'lw':2, 'color':'#A51C30'},
                {'ls':'dotted', 'lw':2, 'color':'#93a1ad'},
                {'ls':'dotted', 'lw':2, 'color':'blue'}
            ],
        contour_args=
            [
                {'ls':'dotted', 'lw':2, 'color':'#A51C30', 'filled' : False},
                {'ls':'dotted', 'lw':2, 'color':'#93a1ad', 'filled' : False},
                {'ls':'dotted', 'lw':2, 'color':'blue', 'filled' : False}
            ],
        )

    #h.set_axes(params=[r'$\tau_{reio}$', r'$A_s$'], lim=[-2, 2, -10, 10])
    #h.set_axes(params=[r'$\tau_{reio}$', r'$h$'], lim=[0.68, 0.72, -10, 10])
    #h.set_axes(params=[r'$\tau_{reio}$', r'$n_s$'], lim=[0.95, 0.99, -10, 10])
    #h.set_axes(params=[r'$\tau_{reio}$', r'$\omega_b$'], lim=[0.020, 0.024, -10, 10])
    #h.set_axes(params=[r'$\tau_{reio}$', r'$\omega_{cdm}$'], lim=[0.108, 0.116, -10, 10])
    
    #for ax in h.subplots[5, 0:4]:
    #    ax.set_ylim(-10, 10)
    
    As_lim = 0.4e-9
    h_lim = 0.013
    ns_lim = 0.01
    omegab_lim = 0.0004
    omegacdm_lim = 0.002
    tau_lim = 10.
    g_lim = 3.
    
    print(mpl.rcParams['agg.path.chunksize'])

    for axidx, axval in enumerate(h.subplots[:,0]):
        print(axidx)
        if axidx==1: 
            axval.set_ylim(fullfid[1]-h_lim, fullfid[1]+h_lim)
            axval.set_xlim(fullfid[0]-As_lim, fullfid[0]+As_lim)
        if axidx==2: 
            axval.set_ylim(fullfid[2]-ns_lim, fullfid[2]+ns_lim)
        if axidx==3: 
            axval.set_ylim(fullfid[3]-omegab_lim, fullfid[3]+omegab_lim)
        if axidx==4: 
            axval.set_ylim(fullfid[4]-omegacdm_lim, fullfid[4]+omegacdm_lim)
        if axidx==5: 
            axval.set_ylim(fullfid[5]-tau_lim, fullfid[5]+tau_lim)
        if axidx==6: 
            axval.set_ylim(fullfid[6]-g_lim, fullfid[6]+g_lim)

    plt.savefig("/Users/nicholasdeporzio/Desktop/ellipse_plots/"+survey+'/contours_M'+str(mval+1)+'.png')
    #plt.show()
    
print('Done')


In [ ]:
plt.get_backend()